# Gemma-3 270m Fine-tuning with Unsloth-MLX

This notebook demonstrates how to fine-tune the Gemma-3 270m model locally on Apple Silicon using `unsloth-mlx`. This tool wraps Apple's MLX framework for efficient Lora fine-tuning.

In [1]:
import os
from dotenv import load_dotenv
from unsloth_mlx import FastLanguageModel
import mlx.core as mx

# Load environment variables
load_dotenv()

model_path = os.getenv("MLX_MODEL_PATH", "./gemma3")
print(f"Loading model from: {model_path}")

Loading model from: ./gemma3


## 1. Load Model and Tokenizer
We load the Gemma-3 270m model. Ensure the files are present in the directory specified above.

In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path,
    max_seq_length = 2048,
    load_in_4bit = True, # Use 4-bit quantization to save memory
)

> *Note:*

> `load_in_4bit = True, # Use 4-bit quantization to save memory`

> It perform QLoRA as Uses 4-bit quantization, allowing you to run models that simply won't fit with standard LoRA.
 For a small model like Gemma-3 270m, both will be extremely fast. However, using QLoRA (4-bit) is "better" for your Mac because:
 It leaves more RAM free for other system tasks.
 It allows you to use longer context lengths (more tokens) without crashing.


## 2. Add LoRA Adapters
LoRA (Low-Rank Adaptation) allows us to fine-tune only a small fraction of the model's parameters.

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
)

LoRA configuration set: rank=16, alpha=16, modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], dropout=0


## 3. Prepare Dataset
We load the locally generated dataset. The path is managed via the `DATASET_PATH` environment variable.

In [ ]:
from datasets import load_dataset

# Load the locally generated dataset from .env path
dataset_file = os.getenv("DATASET_PATH", "dataset.jsonl")
dataset = load_dataset("json", data_files=dataset_file, split="train")

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must match the prompt format expected by Gemma-3
        text = f"### Instruction:\n{instruction}\n\n### Response:\n{output}"
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True,)

print(f"Loaded {len(dataset)} examples from {dataset_file}")
print("Sample training text:")
print(dataset[0]["text"]) 

## 4. Training
Unsloth-MLX provides a simple trainer wrapper.

In [ ]:
from unsloth_mlx import SFTTrainer

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset,
#     dataset_text_field = "text",
#     max_seq_length = 2048,
#     args = {
#         "learning_rate": 2e-4,
#         "num_train_epochs": 1,
#         "logging_steps": 1,
#     },
# )

print("Trainer setup complete. Uncomment the code above to start training.")